<a href="https://colab.research.google.com/github/patience275/recipebot/blob/main/recipie_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain-core  --quiet

In [2]:
!pip install -qU 'langchain[mistralai]'

In [4]:
# 1. Import needed libraries
import os
from langchain_mistralai import ChatMistralAI

# 2. Set your API key (make sure you saved it in Colab beforehand)
os.environ["MISTRAL_API_KEY"] = "MN22VAvXK3a5y0PZggCOx7xJFN4gZ74L"

# 3. Create a simple chat model
llm = ChatMistralAI(model="mistral-tiny")

# 4. Make a test call
response = llm.invoke("Give me a simple 2-ingredient recipe.")
print(response.content)


Sure! Here's a simple recipe for banana oatmeal cookies:

Ingredients:
1. 2 ripe bananas
2. 1 cup of old-fashioned oats

Instructions:
1. Preheat your oven to 350°F (175°C) and line a baking sheet with parchment paper.
2. In a large bowl, mash the ripe bananas until they are smooth.
3. Add the oats to the mashed bananas and mix until well combined.
4. Scoop out spoonfuls of the mixture onto the prepared baking sheet, spacing them about 2 inches apart.
5. Bake for 12-15 minutes, or until the edges are golden brown.
6. Let the cookies cool on the baking sheet for 5 minutes before transferring them to a wire rack to cool completely.

Enjoy your delicious and healthy banana oatmeal cookies!


In [5]:
from langchain.memory import ConversationSummaryMemory
memory=ConversationSummaryMemory(llm=llm, return_messages=True,memory_key='chat_history')

/tmp/ipython-input-346278106.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationSummaryMemory(llm=llm, return_messages=True,memory_key='chat_history')


In [6]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate([
    ('system','you are an excellent chef. using the {ingredients} provided by the user, create a delicious meal. include a catchy name, utensils to use, and the steps to follow to create the meal you suggest'),
    MessagesPlaceholder(variable_name ='chat_history'),
     ('user','{ingredients}')
])

In [7]:
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.runnables import RunnableParallel

chain=RunnableParallel(
    chat_history=lambda x:memory.chat_memory.messages,
    ingredients=RunnablePassthrough() # Change key to 'ingredients' to match prompt
) | prompt | llm

In [13]:
def get_recipie(ingredients):
    response = chain.invoke({"ingredients": ingredients})
    return response.content # Access content using .content

# 5. Test it
user_ingredients = input("Enter the ingredients you have: ")
recipe = get_recipie(user_ingredients)
print("\n🍽️ Here is your recipe suggestion:\n")
print(recipe)

Enter the ingredients you have: beans, rice,tomatoes, oil, onions

🍽️ Here is your recipe suggestion:

Title: Rice 'n' Beans Delight with Tomato Salsa

Utensils:
1. Large pot
2. Medium pot
3. Pan for sautéing
4. Knife
5. Cutting board
6. Measuring cups
7. Measuring spoons
8. Spoon for stirring
9. Serving bowl
10. Plates

Ingredients:
- 1 cup uncooked rice
- 2 cups water (for cooking rice)
- 1 can (15 oz) beans (drained and rinsed)
- 1 medium onion (diced)
- 2 medium tomatoes (diced)
- 2 tbsp oil
- Salt and pepper to taste

Steps to follow:

1. Cook the rice: In a large pot, combine the rice and water. Bring to a boil, then reduce heat to low, cover, and simmer for about 18 minutes or until the rice is tender and the liquid is absorbed. Set aside.

2. Prepare the beans: In a medium pot, heat the oil over medium heat. Add the diced onions and sauté until they become translucent. Add the drained and rinsed beans, season with salt and pepper, and stir to combine. Cook for about 5 minutes.


In [12]:
!pip install bentoml --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.6/199.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.4/118.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.2/452.2 kB 22.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.0.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.7.0 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2025.7.0 which is incompatible.


In [33]:
!pip install -U bentoml --quiet

In [45]:
%%writefile service.py

import bentoml
from bentoml.io import JSON

from langchain_mistralai import ChatMistralAI
from langchain.memory import ConversationSummaryMemory
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.runnables import RunnableParallel

llm = ChatMistralAI(model="mistral-tiny")

memory=ConversationSummaryMemory(llm=llm, return_messages=True,memory_key='chat_history')

prompt=ChatPromptTemplate([
    ('system','you are an excellent chef. using the {ingredients} provided by the user, create a delicious meal. include a catchy name, utensils to use, and the steps to follow to create the meal you suggest'),
    MessagesPlaceholder(variable_name ='chat_history'),
     ('user','{ingredients}')
])

chain=RunnableParallel(
    chat_history=lambda x:memory.chat_memory.messages,
    ingredients=RunnablePassthrough()
) | prompt | llm

def get_recipie(ingredients):
    response = chain.invoke({"ingredients": ingredients})
    return response.content

svc=bentoml.Service('recipie_bot-service')

@svc.apis(input=JSON(),output=JSON())
def recipie(requests):
  ingredients=requests.get('ingredients',"")
  result=get_recipie(ingredients)
  return{'recipie':result}



Overwriting service.py


In [46]:
from google.colab import files

# Replace 'service.py' with your file name
files.download('service.py')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>